In [1]:
from sage.rings.finite_rings.hom_finite_field import FiniteFieldHomomorphism_generic

In [2]:
def get_defining_set(n, pol, field):
    defining_set = []
    
    while len(defining_set) != n:
        aux = field.random_element()
        if pol(aux) != 0 and aux not in defining_set:
            defining_set = defining_set + [aux]
    
    return defining_set

In [3]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``field`` -- finite field on which `self` is defined.
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite field `\GF{p^m}` extending from `field`.
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are not roots of `generating_pol`
    """
    _registered_decoders = {}
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        # TODO: comprobar que el mcd sea 1
        
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()

        if not self._field.is_field() or not self._field.is_finite():
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
        if self.get_dimension() == 0:
            raise ValueError("ERROR. Code dimension is null")
    
    def _repr_(self):
        """
        Representation of a Goppa code.
        """
        return "[{}, {}] Goppa code".format(self._length, self.get_dimension())
    
    def get_generating_pol(self):
        """
        Return the generating polynomial of ``self``.
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set of ``self``.
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial of ``self``.
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of ``self``.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of ``self``.
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def get_dimension(self):
        """
        Return the dimension of the code.
        """
        
        return rank(self.get_generator_matrix())

class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self.code().get_generator_matrix()
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
    
    def _repr_(self):
        """
        Representation of a decoder for a Goppa code
        """
        return "Decoder for {}".format(self.code())
    
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c``: a element of the input space of ``self``.
        """
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):    
            syndrome = syndrome + embFL(c[i])*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword.
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 0
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        secLF = embFL.section()

        # Step 1
        S = self.get_syndrome(word)
        
        if S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t or r_prev.degree() < t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
        
        # Step 5
        # make sigma monic     
        sigma = U_i / U_i.coefficients()[-1]
        
        eta = (-1)^i * r_i / U_i.coefficients()[-1]
        
        # roots of sigma are the locations of the errors
        roots_loc = []
        g = self.get_generating_pol()
        for root in sigma.roots():
            if g(root[0] != 0):
                roots_loc = roots_loc + [self._defining_set.index(root[0])]
                
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(roots_loc)):
            error[roots_loc[i]] = eta.subs(x = sigma.roots()[i][0]) / (sigma_diff.subs(x = sigma.roots()[i][0]))
            
        x = word - vector(field, error)
        
        return x

    def decode_to_message(self, word):
        word = self.decode_to_code(word)
        return self.code().get_generator_matrix().solve_left(word)

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

In [4]:
def get_weight(c):
    w = 0
    
    for elem in c:
        if elem != 0:
            w += 1
    
    return w

In [5]:
class McEliece:
    r"""
    Implementation of McEliece cryptosystem.
    
    INPUT:
    - ``n`` -- 
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are roots of `generating_pol
    """
    def __init__(self, n, q, g):
        """
        Initialize.
        """
        # Goppa code
        F = GF(2)
        L = GF(2^q)
        defining_set = get_defining_set(n, g, L)
        C = Goppa(defining_set, g, F)
        G = C.get_generator_matrix()
        k = G.nrows()
        n = G.ncols()
    
        # random binary nonsingular matrix
        S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
        while rank(S) < k:
            i = randint(0, k-1)
            j = randint(0, k-1)
            S[i,j] = choice(F.list())
        
        # random permutation matrix
        columns = list(range(n))
        P = matrix(F, n)
        
        for i in range(n):
            l = randint(0, len(columns)-1)
            j = columns[l]
            P[i,j] = 1
            columns.remove(j)
        
        self._goppa = C
        self._G = G
        self._t = floor(g.degree()/2)
        self._k = k
        self._n = n
        self._S = S
        self._P = P
        self._public_key = S * self._G * P
        
    def __repr__(self):
        """
        Representation of a McEliece cryptosystem.
        """
        return "McEliece cryptosystem over {}".format(self._goppa)
        
    def get_S(self):
        return self._S
    
    def get_G(self):
        return self._G
    
    def get_P(self):
        return self._P
    
    def get_public_key(self):
        return self._public_key

    def encrypt(self, m):
        """
        Return a chipertext
        
        INPUT:
        - ``m``: a plaintext to encrypt
        """
        # random errors
        e = vector([F(0) for i in range(self._n)])
        
        while get_weight(e) != self._t:
            i = randint(0, len(e) - 1)
            e[i] = choice(F.list())
        
        # chipertext
        c = m * self._public_key + e
        
        print("Error: " + str(e))
        
        return c
    
    def decrypt(self, c):
        word = c * self._P^(-1)
        D = GoppaDecoder(self._goppa)
        word = D.decode_to_message(word)
        message = word * (self._S)^(-1)
        
        return message

In [6]:
def permutation_matrix(x):
    M = matrix(GF(2), len(x), len(x))
    
    for j in range(len(x)):
        i = x[j] - 1
        M[i,j] = 1
    
    return M

### n = 64

In [26]:
q = 8
n = 64
t = 4
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]

min_fitness = floor(t/2)

while True:
    g = R.random_element(t)
    
    if g.is_monic():
        break
g

x^4 + (z8^6 + z8^3 + z8)*x^3 + (z8^7 + z8^6 + z8^3)*x^2 + (z8^7 + z8^6 + z8^4 + z8 + 1)*x + z8^7 + z8^6

### n = 128

In [98]:
q = 8
n = 128
t = 8
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]

min_fitness = floor(t/2)

while True:
    g = R.random_element(t)
    
    if g.is_monic():
        break
g

x^8 + (z8^6 + z8^5)*x^7 + (z8^5 + z8^2 + z8 + 1)*x^6 + (z8^7 + z8^6 + z8^5 + z8^2 + z8 + 1)*x^5 + z8^3*x^4 + (z8^6 + z8^5 + z8^2 + 1)*x^3 + (z8^6 + z8^5 + z8^2 + 1)*x^2 + (z8^5 + z8)*x + z8^7 + z8^6 + z8^2 + 1

### n = 256

In [123]:
q = 9
n = 256
t = 14
F = GF(2)
L = GF(2^q)
a = L.gen()
R.<x> = L[]

min_fitness = floor(t/2)

while True:
    g = R.random_element(t)
    
    if g.is_monic():
        break
g

x^14 + (z9^6 + z9^5 + z9^2 + 1)*x^13 + (z9^8 + z9^7 + z9^6 + z9)*x^12 + (z9^5 + z9^3 + 1)*x^11 + (z9^8 + z9^4 + z9^2 + z9)*x^10 + (z9^8 + z9^7 + z9^6 + z9^3 + 1)*x^9 + (z9^8 + z9^7 + z9^3)*x^8 + z9^6*x^7 + (z9^7 + z9^5 + z9^4 + z9)*x^6 + (z9^8 + z9^6 + z9^5 + z9^4 + z9^2 + 1)*x^5 + (z9^7 + z9^6)*x^4 + (z9^6 + z9^5 + z9 + 1)*x^3 + (z9^8 + z9^7 + z9^6 + z9^4 + z9 + 1)*x^2 + (z9^8 + z9^6 + z9^4 + z9^3 + 1)*x + z9^8 + z9^6 + z9 + 1

In [124]:
ME = McEliece(n, q, g)
ME

McEliece cryptosystem over [256, 130] Goppa code

In [100]:
G = ME.get_public_key()
#print(G)

In [101]:
public_key = ME.get_public_key()
message = []
while len(message) != public_key.nrows():
    message = message + [choice(F.list())]

message = vector(F, message)

print("\nMessage to encrypt:")
print(message)

encrypted_message = ME.encrypt(message)

print("\nEncrypted message:")
print(encrypted_message)


Message to encrypt:
(1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0)
Error: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

Encrypted message:
(0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0

In [102]:
G = G.stack(encrypted_message)
#print(G)

# GGA

In [31]:
def fitness_vector(v):
    f = 0
    
    for i in v:
        if i != 0:
            f += 1
    
    return f

In [32]:
def fitness_matrix(M):
    for i in M:
        f_min = M.ncols()
        f = fitness_vector(i)
        
        if f < f_min:
            f_min = f
    
    return f_min

In [33]:
def fitness_permutation(G, c):
    n = G.nrows()
    m = G.ncols()
    Gx = matrix(F, n, m)
    for i in range(0, m):
        col = c[i] - 1

        for j in range(0, n):
            Gx[j,col] = G[j,i]

    return fitness_matrix(Gx.rref())

In [34]:
def random_sol():
    tam = G.ncols()
    sol = []
    
    while len(sol) != tam:
        r = randint(1, tam)
        
        if r not in sol:
            sol = sol + [r]
            
    return sol

In [35]:
def crossover(p1, p2):
    return Permutation(p1) * Permutation(p2), Permutation(p2) * Permutation(p1)

In [36]:
def mutation(p):
    n = len(p)
    k = randint(0, n - 1)
    k1 = randint(0, k - 1)
    k2 = randint(k, n - 1)

    return Permutation(k1, k2) * p

In [37]:
def valid(c):
    return c != 0

In [38]:
def GGA(N, pc, MaxReinit, min_fitness):
    t = 0
    Maxt = 500000
    #Maxt = 10000
    Pt = []
    reinit = 0
    
    # Initialize the Population P(t)
    for i in range(0, N):
        sol = random_sol()
        
        while not valid(sol):
            sol = random_sol()
            
        Pt = Pt + [sol]
    
    Pt = Matrix(Pt)
    
    # Evaluate
    fitnessPt = []
    for Pti in Pt:
        fitnessPt = fitnessPt + [fitness_permutation(G, Pti)]
        
    while(t < Maxt and min(fitnessPt) > min_fitness):
        Ptsig = []
        fitnessPtsig = []
        
        # Binary tournament selection
        parents = []
        for i in range(0, N):            
            ind1 = Pt[randint(0, N-1)]
            ind2 = Pt[randint(0, N-1)]
            
            if (fitness_permutation(G, ind1) >= fitness_permutation(G, ind2)):
                parents = parents + [ind2]
            else:
                parents = parents + [ind1]
        
        parents = Matrix(parents)
        
        # Generation
        for i in range(0, N/2):
            nCrossovers = pc * N/2 # number of crossovers
            
            if nCrossovers > 0: # TODO: es uniforme?
                c1, c2 = crossover(parents[2*i], parents[2*i + 1])
                nCrossovers -= 1
            else:
                c1 = mutation(copy(parents[2*i]))
                c2 = mutation(copy(parents[2*i + 1]))
            
            # Valid
            while not valid(c1):
                c1 = random_sol()
            while not valid(c2):
                c2 = random_sol()
            
            # Update
            Ptsig = Ptsig + [c1]
            Ptsig = Ptsig + [c2]
        
        Ptsig = Matrix(Ptsig)
        
        # Evaluate
        for Ptsigi in Ptsig:
            fitnessPtsig = fitnessPtsig + [fitness_permutation(G, Ptsigi)]
        
        # No improvement
        if min(fitnessPtsig) > min(fitnessPt):
            worst = fitnessPtsig.index(max(fitnessPtsig))
            best = fitnessPt.index(min(fitnessPt))
            Ptsig[worst] = Pt[best]
            fitnessPtsig[worst] = fitnessPt[best]
            reinit += 1
        else:
            reinit = 0
        
        # Restart
        if reinit >= MaxReinit:
            Ptsig = []
            for i in range(0, N-1):
                Ptsig = Ptsig + [random_sol()]
            
            best = fitnessPt.index(min(fitnessPt))
            Ptsig = Ptsig + [Pt[best]]
            Ptsig = Matrix(Ptsig)
            
            # Evaluate
            fitnessPtsig = []
            for Ptsigi in Ptsig:
                fitnessPtsig = fitnessPtsig + [fitness_permutation(G, Ptsigi)]
        
        fitnessPt = fitnessPtsig
        Pt = Ptsig
        t += 1
    
    best = fitnessPt.index(min(fitnessPt))
    
    return Pt[best], fitness_permutation(G, Pt[best])

In [103]:
N = 50
n = 0

while n != N:
    t += 1
    c, f = GGA(400, 0.7, 100000, min_fitness)
    P = permutation_matrix(c)
    M = (G*P).rref()
    v = [0] * M.ncols()
    for i in range(M.ncols()):
        v[i] = M[M.nrows()-1, i]
    v = vector(GF(2), v)
    e = v*P^(-1)
    z = encrypted_message - e
    G2 = G.delete_rows([G.nrows()-1])
    
    try:
        m = G2.solve_left(z) # z = mG
        
        if message == m:
            break
    except(ValueError):
        continue

print(n)
print("m: " + str(m))

11
m: (1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0)


# CHC

In [90]:
def distance(x, y):
    assert(len(x) == len(y))
    
    n = len(x)
    count = 0
    for i in range(0, n):
        if x[i] != y[i]:
            count += 1
            
    return count

In [91]:
def distanceP(P, tau):
    d = 0
    Maxdist = 0
    count = 0
    for i in range(0, P.nrows()):
        for j in range(i, P.nrows()):
            dist = distance(P[i], P[j])
            
            if dist > Maxdist:
                Maxdist = dist
            
            d += dist
            count += 1
    
    d = d * 1.0/count
    dec = tau * Maxdist
    
    return d, dec

In [92]:
def CHC(N, tau, min_fitness):
    t = 0
    #Maxt = 500000
    Maxt = 10000
    Pt = []
    reinit = 0
    
    # Initialize the Population P(t)
    for i in range(0, N):
        sol = random_sol()
        
        while not valid(sol):
            sol = random_sol()
            
        Pt = Pt + [sol]
    
    Pt = Matrix(Pt)
    
    # Evaluate
    fitnessPt = []   
    for Pti in Pt:
        fitnessPt = fitnessPt + [fitness_permutation(G, Pti)]
    
    # Distance
    d, dec = distanceP(Pt, tau)
        
    while(t < Maxt and min(fitnessPt) > min_fitness):
        Ct = []
        Ptsig = []
        fitnessCt = []
        fitnessPtsig = []
        
        # Parent selection
        parents = []
        for i in range(0, N):            
            p = randint(0, N-1)
            parents = parents + [Pt[p]]
        
        parents = Matrix(parents)
        
        # Generation
        for i in range(0, N/2):            
            if distance(parents[2*i], parents[2*i + 1]) < d:
                c1, c2 = crossover(parents[2*i], parents[2*i + 1])
                Ct = Ct + [c1]
                Ct = Ct + [c2]
        
        Ct = Matrix(Ct)
        
        # Evaluate
        for Cti in Ct:
            fitnessCt = fitnessCt + [fitness_permutation(G, Cti)]
        
        if Ct.nrows() == 0:
            Ptsig = Pt
            fitnessPtsig = fitnessPt
        else:
            fitnessPtaux = copy(fitnessPt)
            fitnessCtaux = copy(fitnessCt)
            
            for i in range(0, N):
                bestPt = min(fitnessPtaux)
                bestCt = min(fitnessCtaux)

                if bestPt < bestCt:
                    index = fitnessPtaux.index(bestPt)
                    Ptsig = Ptsig + [Pt[index]]
                    fitnessPtsig = fitnessPtsig + [bestPt]
                    fitnessPtaux[index] = Infinity
                else:
                    index = fitnessCtaux.index(bestCt)
                    Ptsig = Ptsig + [Ct[index]]
                    fitnessPtsig = fitnessPtsig + [bestCt]
                    fitnessCtaux[index] = Infinity
        
            Ptsig = Matrix(Ptsig)
        
        # No improvement
        if Pt == Ptsig:
            d = d - dec
            
            if d <= 0:
                Ptsig = []
                for i in range(0, N-1):
                    Ptsig = Ptsig + [random_sol()]

                best = fitnessPt.index(min(fitnessPt))
                Ptsig = Ptsig + [Pt[best]]
                Ptsig = Matrix(Ptsig)
                
                # Distance
                d, dec = distanceP(Pt, tau)
        
        fitnessPt = fitnessPtsig
        Pt = Ptsig
        t += 1
    
    best = fitnessPt.index(min(fitnessPt))
    
    return Pt[best], fitness_permutation(G, Pt[best])

In [104]:
N = 50
n = 0

while n != N:
    n += 1
    c, f = CHC(400, 0.8, min_fitness)
    P = permutation_matrix(c)
    M = (G*P).rref()
    v = [0] * M.ncols()
    for i in range(M.ncols()):
        v[i] = M[M.nrows()-1, i]
    v = vector(GF(2), v)
    e = v*P^(-1)
    z = encrypted_message - e
    G2 = G.delete_rows([G.nrows()-1])
    
    try:
        m = G2.solve_left(z) # z = mG
        
        if message == m:
            break
    except(ValueError):
        continue

print(n)
print("m: " + str(m))

24
m: (1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0)
